# Домашняя работа к уроку 2. Профилирование пользователей. Сегментация аудитории: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)

1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

3. Повторить п.2, но используя уже не медиану, а max

4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.

5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных


**Ссылки**
- http://www.machinelearning.ru/wiki/images/d/d5/Voron17survey-artm.pdf

- https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation

#### Выполнение

##### 2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. 
Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

In [1]:
import pandas as pd

In [2]:
# Загрузим новости, пользователей пользователей и списки последних прочитанных новостейnews = pd.read_csv("./articles.csv")
news = pd.read_csv("articles.csv")
users = pd.read_csv("users_articles.csv")

In [3]:
print(news.shape)
print(users.shape)

(27000, 2)
(8000, 2)


In [4]:
display(news.head(3), users.head(3))

,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [5]:
# Получаем векторные представления новостей
"Gensim — библиотека обработки естественного языка предназначения для «Тематического моделирования»."
!pip install --upgrade gensim

Requirement already up-to-date: gensim in /Users/alenakukhta/opt/anaconda3/lib/python3.8/site-packages (4.1.2)


In [6]:
"""
razdel - rule-based system for Russian sentence and word tokenization. See natasha.github.io article for more info.
https://github.com/natasha/razdel"""
!pip install razdel

In [7]:
"Морфологический анализатор pymorphy2"
!pip install pymorphy2

In [8]:
!pip install nltk

In [9]:
!pip install spacy

In [10]:
!pip install natasha

In [11]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [12]:
#предобработка текстов
import re
import numpy as np
from razdel import tokenize  # https://github.com/natasha/razdel
import pymorphy2

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alenakukhta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
stopword_ru = nltk.corpus.stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [14]:
with open('./stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [15]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [16]:
%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-15-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.1 µs


In [17]:
%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


In [18]:
# Обучение модели
%time
# сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 2.86 µs


In [19]:
#словарь слов
print([common_dictionary[i] for i in range(10, 15)])

['ватутин', 'взаимодействие', 'власть', 'войти', 'вячеслав']


In [20]:
# запуск обучения
%time
from gensim.models import LdaModel

# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary, passes=5)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [21]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [22]:
# Анализ тем
# Create a new corpus, made of previously unseen documents.
# Создайте новый корпус, состоящий из ранее невидимых документов.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]

print(other_texts[2])
lda[unseen_doc]

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(1, 0.19171016),
 (3, 0.06683121),
 (4, 0.03085962),
 (10, 0.041709185),
 (14, 0.37511405),
 (15, 0.098862864),
 (16, 0.1281985),
 (24, 0.049202584)]

In [23]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: китай остров китайский японский япония южный употребление
topic_1: опубликовать источник новый статья наука первый автор
topic_2: ребёнок женщина мужчина дом семья сотрудник врач
topic_3: фонд погибнуть аэропорт рейс трагедия кризис произойти
topic_4: рубль рост рынок млн цена тыс уровень
topic_5: исследование мозг возраст пенсия расследование метод лаборатория
topic_6: украина украинский киев эксперимент задержать полицейский украинец
topic_7: земля двигатель поверхность научный океан блок скорость
topic_8: планета активность праздничный ирак рот столетие роды
topic_9: россия млрд проект российский правительство решение эксперт
topic_10: закон система доход документ средство мера право
topic_11: день житель станция центр москва час около
topic_12: nn россия глава экономика владимир российский путин
topic_13: банк уголовный налог конкурс признать преступление бывший
topic_14: мероприятие сайт участник игра команда организатор узнать
topic_15: всё очень большой первый научный с

In [24]:
# функция, которая будет нам возвращать векторное представление новости
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(
        zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [25]:
# вектора новостей
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.021271,0.016387,0.000000,0.000000,0.000000,0.0,0.0,0.011827,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.018461
1,4896,0.000000,0.000000,0.216502,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.305432,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,4897,0.000000,0.191681,0.000000,0.066822,0.030851,0.0,0.0,0.000000,0.0,...,0.098947,0.128217,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.049196
3,4898,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.014330,0.0,...,0.410483,0.084163,0.000000,0.0,0.073694,0.0,0.0,0.0,0.0,0.000000
4,4899,0.052513,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [26]:
# векторные представления пользователей
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [27]:
user_articles_list = users['articles'].iloc[33]
display(user_articles_list, 
        eval(user_articles_list)
       )

'[323329, 321961, 324743, 323186, 324632, 474690]'

[323329, 321961, 324743, 323186, 324632, 474690]

In [28]:
def get_user_embedding_mean(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

def get_user_embedding_median(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

In [29]:
def score_with_user_embedding(func):
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: func(x), 1)])
    user_embeddings.columns = [f'topic_{i}' for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid'] + [f'topic_{i}' for i in range(25)]]

    X = pd.merge(user_embeddings, target, 'left')

    #разделим данные на train/test
    X_train, X_test, y_train, y_test = train_test_split(
    X[[f'topic_{i}' for i in range(25)]], X['churn'], random_state=0)
    
    logreg = LogisticRegression()

    #обучим наш пайплайн
    logreg.fit(X_train, y_train)

    #наши прогнозы для тестовой выборки
    preds = logreg.predict_proba(X_test)[:, 1]

    # Рассчитаем Precision, Recall, F_score, roc_auc_score
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    roc_auc_score = roc_auc_score(y_test, preds)

    return round(thresholds[ix], 4), round(fscore[ix], 4), round(precision[ix], 4), round(recall[ix], 4), round(roc_auc_score, 4)

In [30]:
target = pd.read_csv("users_churn.csv")

#### 5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

In [31]:
results = pd.DataFrame(np.array([
    score_with_user_embedding(func=get_user_embedding_mean),
    score_with_user_embedding(func=get_user_embedding_median),
    score_with_user_embedding(func=get_user_embedding_max)
]), columns=['Best Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC score'])

results['func'] = ['mean', 'median', 'max']
results = results.set_index('func')

In [32]:
results

,Best Threshold,F-Score,Precision,Recall,ROC AUC score
func,,,,,
mean,0.2946,0.8280,0.8118,0.8449,0.9802
median,0.2593,0.8260,0.7770,0.8816,0.9822
max,0.3552,0.7665,0.7500,0.7837,0.9725


#### 6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

Cамый эффективный способ - через max. Вероятнее всего из-за минимизации разброса интересов пользователя.